In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
# Database Reading and Manipulation
import pandas as pd

# Linear Algebra
import numpy as np

# Plotting
import matplotlib.pyplot as plt

# Processing results
import json

# Model Selection
from sklearn.model_selection import train_test_split
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import GridSearchCV

# Modeling
from sklearn.linear_model import LinearRegression

# Processing
from sklearn.preprocessing import StandardScaler

# Data imputation
from sklearn.impute import SimpleImputer

# Pipeline
from sklearn.pipeline import Pipeline

# Metrics
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import r2_score


# Custom modules
## Model selection
from src.cross_validation.blocking_time_series_split import BlockingTimeSeriesSplit

## Function to print scores
from src.utils.print_scores import print_scores

## Function to calculate score regression metrics
from src.utils.score_regression_metrics import score_regression_metrics

## Function to fill the results metric dict
from src.utils.fill_results_dict import fill_results_dict

<IPython.core.display.Javascript object>

# Functions and definitions

In [3]:
index_to_save = 4

<IPython.core.display.Javascript object>

In [4]:
SEED = 47
METRICS = (
    "neg_root_mean_squared_error",
    "neg_mean_absolute_error",
    "neg_mean_absolute_percentage_error",
    "r2",
)
METRICS_DICT = {
    "neg_root_mean_squared_error": "RMSE",
    "neg_mean_absolute_error": "MAE",
    "neg_mean_absolute_percentage_error": "MAPE",
    "r2": "R2",
}

<IPython.core.display.Javascript object>

## Defining a dataframe structure to save the results

In [5]:
results_to_save = []

results_dict = {
    "Category": "Local Model",
    "Company": "partner_iv",
    "Features": "Chemical + Mineralogical + Feature Engineering",
    "Data Shape": None,
    "Timesteps": None,
    "Model": "Linear Regression",
    "Model Params": None,
    "Scaler": "Standard Scaler",
    "Scaler Params": None,
    "Imputer": "Median",
    "Imputer Params": None,
    "Cross Validation": None,
    "Cross Validation Params": np.nan,
    "RMSE Train": np.nan,
    "MAE Train": np.nan,
    "MAPE Train": np.nan,
    "R2 Train": np.nan,
    "RMSE Test": np.nan,
    "MAE Test": np.nan,
    "MAPE Test": np.nan,
    "R2 Test": np.nan,
}

<IPython.core.display.Javascript object>

# Reading the dataset

In [6]:
df = pd.read_csv("../../../../../data/processed/partner_iv/cem_a.csv")

<IPython.core.display.Javascript object>

## Defining Features

In this set of experiments we keep only chemical and mineralogical features yielded by the same testing method/procedure

In [7]:
df_copy = df.copy()
df_copy = df.drop(
    [
        # Properties
        "Blaine",
        "ph2oimm",  # Maybe pH of the immersion liquid | pH (acidity or alkalinity)
        "Initial Setting Time",
        "flow",  # Maybe flow table test
        "residuo 24 micron",  # Maybe the residue left after passing the material through a sieve
        "R_wp",  # Maybe water to powder ratio
        "CS2",  # 2-day Compressive Strength
        "Soundness",
    ],
    axis=1,
).copy()

<IPython.core.display.Javascript object>

## Feature Engineering

In [8]:
# Feature Engineering over Chemical Features
ch_features = ["CaO", "Ca(OH)2", "Periclasio (MgO)", "K2SO4", "Cl-", "SO3"]
df_copy["std_ch_feats"] = df_copy[ch_features].std(ddof=0, axis=1)

# df_copy["ratio_MgO_to_CaO"] = df_copy["Periclasio (MgO)"] / df_copy["CaO"]

# Feature Engineering over Mineralogical Features
mi_features_set1 = [
    "Alite_M1 C3S M1",
    "Alite_M3 C3S M3",
    "Anidrite",
    "Aphthitalite – (K,Na)3(SO4)2",
    "Belite_beta",
    "C3A cub",
    "C3A tot",
    "C3A_ortho",
    "C3S_CS (taglia dei cristalliti C3S)",
    "C4AF",
    "CO2_XRD",
    "Calce libera",
    "Calcite – CaCO3",
    "Emiidrato",
    "Gesso",
    "Langbeinite – MgK2(SO4)2",
    "Quartz",
    "SO3_XRD",
]

df_copy["std_mi_set1_feats"] = df_copy[mi_features_set1].std(ddof=0, axis=1)

<IPython.core.display.Javascript object>

# 1. Linear Regression

<h2>1. Dataset: df_no_cs</h2> <br>In this dataset the CS1, CS3  and CS7 variables are not considered. Only Chemical and mineralogical features measured by the same method. For this particular dataset, all chemical features, with the exception of LOI were measured by XRF and XRD methods.

In [9]:
y = df_copy.pop("CS28").values
x = df_copy.drop(["Date"], axis=1)
dates = df["Date"].copy()

<IPython.core.display.Javascript object>

## 1.1 Repeated KFold Cross validation

<b>Dataset shape:</b> (1234, 38)<br>
<b>Repeats:</b>10<br>
<b>Splits:</b>10<br>
    1. 10 folds of 123 samples each
    2. 90% train (1111 samples each fold)
    3. 10% test (123 samples each fold)
<b>Total:</b> 100 models<br>

In [10]:
repeats = 3
n_splits = 5
pipeline = Pipeline(
    [
        ("imputer", SimpleImputer(strategy="median")),
        ("transformer", StandardScaler()),
        ("estimator", LinearRegression()),
    ]
)
cv = RepeatedKFold(n_splits=n_splits, n_repeats=repeats, random_state=SEED)
scores = cross_validate(
    pipeline,
    x,
    y,
    scoring=METRICS,
    cv=cv,
    n_jobs=-1,
    return_train_score=True,
)
print("Repeated Cross Validation:")
print(f"Repeats: {repeats}")
print(f"n_splits: {n_splits}")
print()
print_scores(scores, METRICS, METRICS_DICT)

results_dict_copy = results_dict.copy()
results_dict_copy["Cross Validation"] = "Repeated KFold"
results_dict_copy["Cross Validation Params"] = '{"N_Splits": 5, "Repeats": 3}'
results_dict_copy["Data Shape"] = x.shape
df_results = fill_results_dict(results_dict_copy, scores)
results_to_save.append(df_results)

Repeated Cross Validation:
Repeats: 3
n_splits: 5

******
[TRAIN]
******
RMSE: -2.521 (0.055)
MAE: -2.030 (0.043)
MAPE: -0.038 (0.001)
R2: 0.445 (0.019)


******
[TEST]
******
RMSE: -2.993 (0.355)
MAE: -2.398 (0.223)
MAPE: -0.044 (0.004)
R2: 0.187 (0.121)




<IPython.core.display.Javascript object>

In [11]:
pd.concat(results_to_save).reset_index().groupby(
    ["Features", "Model", "Cross Validation"], dropna=False
)[["RMSE Test", "MAE Test", "MAPE Test", "R2 Test"]].agg(
    ["mean", lambda series: pd.Series(series.std(ddof=0), name="std")]
).reset_index().rename(
    columns={"<lambda_0>": "std"}
)

Features              Model  \
                                                                       
0  Chemical + Mineralogical + Feature Engineering  Linear Regression   

  Cross Validation RMSE Test            MAE Test           MAPE Test  \
                        mean       std      mean       std      mean   
0   Repeated KFold  2.993266  0.355498  2.398074  0.223149  0.044482   

              R2 Test            
        std      mean       std  
0  0.004289  0.187197  0.121415

<IPython.core.display.Javascript object>

## 1.2. Blocking Time Series Cross Validation

<b>Dataset shape:</b> (1234, 38)<br>
<b>Splits:</b>5<br>    
    1. 5 folds of 246 samples
    2. 50% train (123 samples each fold)
    3. 50% test (123 samples each fold)
<b>Total:</b> 5 models<br>

In [12]:
n_splits = 5
train_size = 0.8

pipeline = Pipeline(
    [
        ("imputer", SimpleImputer(strategy="median")),
        ("transformer", StandardScaler()),
        ("estimator", LinearRegression()),
    ]
)
cv = BlockingTimeSeriesSplit(n_splits=n_splits, train_size=train_size)
scores = cross_validate(
    pipeline,
    x,
    y,
    scoring=METRICS,
    cv=cv,
    n_jobs=-1,
    return_train_score=True,
)
print("Blocking Time Series Split:")
print(f"Repeats: {repeats}")
print(f"n_splits: {n_splits}")
print()
print_scores(scores, METRICS, METRICS_DICT)

results_dict_copy = results_dict.copy()
results_dict_copy["Cross Validation"] = "Blocking Time Series Split"
results_dict_copy[
    "Cross Validation Params"
] = '{"N_Splits": 5, "Repeats": 1, "train_size": 0.8}'
results_dict_copy["Data Shape"] = x.shape
df_results = fill_results_dict(results_dict_copy, scores)
results_to_save.append(df_results)

Blocking Time Series Split:
Repeats: 3
n_splits: 5

******
[TRAIN]
******
RMSE: -1.272 (0.369)
MAE: -0.982 (0.249)
MAPE: -0.018 (0.005)
R2: 0.801 (0.067)


******
[TEST]
******
RMSE: -4.746 (1.243)
MAE: -3.534 (0.329)
MAPE: -0.065 (0.006)
R2: -4.030 (2.742)




<IPython.core.display.Javascript object>

In [13]:
pd.concat(results_to_save).reset_index().groupby(
    ["Features", "Model", "Cross Validation"], dropna=False
)[["RMSE Test", "MAE Test", "MAPE Test", "R2 Test"]].agg(
    ["mean", lambda series: pd.Series(series.std(ddof=0), name="std")]
).reset_index().rename(
    columns={"<lambda_0>": "std"}
)

Features              Model  \
                                                                       
0  Chemical + Mineralogical + Feature Engineering  Linear Regression   
1  Chemical + Mineralogical + Feature Engineering  Linear Regression   

             Cross Validation RMSE Test            MAE Test            \
                                   mean       std      mean       std   
0  Blocking Time Series Split  4.745936  1.242746  3.534024  0.328948   
1              Repeated KFold  2.993266  0.355498  2.398074  0.223149   

  MAPE Test             R2 Test            
       mean       std      mean       std  
0  0.065199  0.005893 -4.029997  2.742172  
1  0.044482  0.004289  0.187197  0.121415

<IPython.core.display.Javascript object>

## 1.3. Time Series Split Cross Validation

The training set has size i * n_samples // (n_splits + 1) + n_samples % (n_splits + 1) in the i th split, with a test set of size n_samples//(n_splits + 1) by default, where n_samples is the number of samples.


<b>Dataset shape:</b> (1234, 38)<br>
<b>Splits:</b>10<br>    
    1. Train: 10 folds of 114, 226, 338, 450, 562, 675, 787, 899, 1011, 1123 samples each fold
    2. Test: 112 samples each fold
<b>Total:</b> 10 models<br>

In [14]:
n_splits = 5
gap = 0
pipeline = Pipeline(
    [
        ("imputer", SimpleImputer(strategy="median")),
        ("transformer", StandardScaler()),
        ("estimator", LinearRegression()),
    ]
)
cv = TimeSeriesSplit(gap=gap, max_train_size=None, n_splits=n_splits, test_size=None)

scores = cross_validate(
    pipeline,
    x,
    y,
    scoring=METRICS,
    cv=cv,
    n_jobs=-1,
    return_train_score=True,
)
print("Time Series Split:")
print(f"Repeats: {repeats}")
print(f"n_splits: {n_splits}")
print()
print_scores(scores, METRICS, METRICS_DICT)

results_dict_copy = results_dict.copy()
results_dict_copy["Cross Validation"] = "Time Series Split"
results_dict_copy["Cross Validation Params"] = '{"N_Splits": 5, "Repeats": 1, "Gap": 0}'
results_dict_copy["Data Shape"] = x.shape
df_results = fill_results_dict(results_dict_copy, scores)
results_to_save.append(df_results)

Time Series Split:
Repeats: 3
n_splits: 5

******
[TRAIN]
******
RMSE: -2.264 (0.186)
MAE: -1.725 (0.204)
MAPE: -0.032 (0.004)
R2: 0.609 (0.061)


******
[TEST]
******
RMSE: -4.284 (1.156)
MAE: -3.103 (0.417)
MAPE: -0.057 (0.009)
R2: -2.014 (1.679)




<IPython.core.display.Javascript object>

In [15]:
pd.concat(results_to_save).groupby(["Features", "Model", "Cross Validation"])[
    ["RMSE Test", "MAE Test", "MAPE Test", "R2 Test"]
].agg(
    ["mean", lambda series: pd.Series(series.std(ddof=0), name="std")]
).reset_index().rename(
    columns={"<lambda_0>": "std"}
)

Features              Model  \
                                                                       
0  Chemical + Mineralogical + Feature Engineering  Linear Regression   
1  Chemical + Mineralogical + Feature Engineering  Linear Regression   
2  Chemical + Mineralogical + Feature Engineering  Linear Regression   

             Cross Validation RMSE Test            MAE Test            \
                                   mean       std      mean       std   
0  Blocking Time Series Split  4.745936  1.242746  3.534024  0.328948   
1              Repeated KFold  2.993266  0.355498  2.398074  0.223149   
2           Time Series Split  4.283606  1.156246  3.103097  0.416686   

  MAPE Test             R2 Test            
       mean       std      mean       std  
0  0.065199  0.005893 -4.029997  2.742172  
1  0.044482  0.004289  0.187197  0.121415  
2  0.057231  0.008904 -2.013880  1.678693

<IPython.core.display.Javascript object>

## 1.4. Out of time Split Cross Validation

<b>Dataset shape:</b> (1234, 38)<br>
<b>Train size: 80%</b><br>
<b>Test  size: 20%</b>


<b>Splits:</b> 2<br>    
    1. Train: 987
    2. Test: 247
<b>Total:</b> 1 model<br>

In [16]:
test_size = 0.2

x_train, x_test, y_train, y_test = train_test_split(
    x, y, test_size=test_size, random_state=SEED, shuffle=False
)
pipeline = Pipeline(
    [
        ("imputer", SimpleImputer(strategy="median")),
        ("transformer", StandardScaler()),
        ("estimator", LinearRegression()),
    ]
)

pipeline.fit(x_train, y_train)

y_train_pred = pipeline.predict(x_train)
y_test_pred = pipeline.predict(x_test)

scores = score_regression_metrics(y_train, y_train_pred, y_test, y_test_pred)
print_scores(scores, METRICS, METRICS_DICT)

results_dict_copy = results_dict.copy()
results_dict_copy["Cross Validation"] = "Out of time Split"
results_dict_copy["Cross Validation Params"] = '{"Test Size": 0.2}'
results_dict_copy["Data Shape"] = x.shape
df_results = fill_results_dict(
    results_dict_copy, {key: [value] for key, value in scores.items()}
)
results_to_save.append(df_results)

******
[TRAIN]
******
RMSE: 2.430 (0.000)
MAE: 1.921 (0.000)
MAPE: 0.035 (0.000)
R2: 0.533 (0.000)


******
[TEST]
******
RMSE: 3.881 (0.000)
MAE: 3.098 (0.000)
MAPE: 0.059 (0.000)
R2: -1.755 (0.000)




<IPython.core.display.Javascript object>

In [17]:
pd.concat(results_to_save).groupby(["Features", "Model", "Cross Validation"])[
    ["RMSE Test", "MAE Test", "MAPE Test", "R2 Test"]
].agg(
    ["mean", lambda series: pd.Series(series.std(ddof=0), name="std")]
).reset_index().rename(
    columns={"<lambda_0>": "std"}
)

Features              Model  \
                                                                       
0  Chemical + Mineralogical + Feature Engineering  Linear Regression   
1  Chemical + Mineralogical + Feature Engineering  Linear Regression   
2  Chemical + Mineralogical + Feature Engineering  Linear Regression   
3  Chemical + Mineralogical + Feature Engineering  Linear Regression   

             Cross Validation RMSE Test            MAE Test            \
                                   mean       std      mean       std   
0  Blocking Time Series Split  4.745936  1.242746  3.534024  0.328948   
1           Out of time Split  3.881371  0.000000  3.097853  0.000000   
2              Repeated KFold  2.993266  0.355498  2.398074  0.223149   
3           Time Series Split  4.283606  1.156246  3.103097  0.416686   

  MAPE Test             R2 Test            
       mean       std      mean       std  
0  0.065199  0.005893 -4.029997  2.742172  
1  0.058789  0.000000 -1.754612  0.000000  
2  0.044482  0.004289  0.187197  0.121415  
3  0.057231  0.008904 -2.013880  1.678693

<IPython.core.display.Javascript object>

In [18]:
pipeline.named_steps["estimator"].coef_

array([ 6.21395108e-01, -2.75301062e-01, -4.89577440e+01, -3.96179380e+01,
        1.70538460e+01, -4.99142320e-01, -5.12301088e+01, -3.38273405e+00,
       -1.92396301e+00, -3.46099843e+00,  3.36565469e+00, -9.80679302e-01,
       -8.75632410e+00, -1.44275822e+01,  1.35033185e+01, -3.87158077e-01,
        1.09522582e-02, -1.75874222e-01,  2.55694522e-01, -9.29657959e-01,
       -5.32109459e-02,  6.51205212e-01,  1.37312067e+00,  2.14664552e+01,
       -5.16721431e-01, -2.37624470e+01, -4.06515694e-01,  5.14088277e+01])

<IPython.core.display.Javascript object>

In [19]:
pipeline.named_steps["estimator"].intercept_

54.571031746031885

<IPython.core.display.Javascript object>

In [20]:
coeff = pipeline.named_steps["estimator"].coef_
np.array(x.columns)[coeff == 0]

array([], dtype=object)

<IPython.core.display.Javascript object>

In [21]:
x.drop(np.array(x.columns)[coeff == 0], axis=1).columns

Index(['SO3', 'Cl-', 'Alite_M3 C3S M3', 'Alite_M1 C3S M1', 'Alite_Sum C3S tot',
       'Ratio_M1 (rapporto M1/M3)', 'C3S_CS (taglia dei cristalliti C3S)',
       'Belite_beta', 'C3A cub', 'C3A_ortho', 'C3A tot', 'C4AF', 'CaO',
       'Ca(OH)2', 'Calce libera', 'Periclasio (MgO)', 'Quartz', 'K2SO4',
       'Langbeinite – MgK2(SO4)2', 'Aphthitalite – (K,Na)3(SO4)2', 'Gesso',
       'Emiidrato', 'Anidrite', 'Calcite – CaCO3', 'SO3_XRD', 'CO2_XRD',
       'std_ch_feats', 'std_mi_set1_feats'],
      dtype='object')

<IPython.core.display.Javascript object>

In [22]:
x.drop(np.array(x.columns)[coeff == 0], axis=1).columns.shape

(28,)

<IPython.core.display.Javascript object>

In [23]:
x.columns.shape

(28,)

<IPython.core.display.Javascript object>

In [24]:
coeffs = pd.DataFrame(
    {col: [c] for col, c in zip(x.columns, coeff)}, index=["Coefficients"]
)

<IPython.core.display.Javascript object>

In [25]:
coeffs.T["Coefficients"].sort_values(ascending=False).to_frame(
    name="Coefficients"
).style.background_gradient(axis=None, vmin=1, vmax=5, cmap="Greens")

,Coefficients
std_mi_set1_feats,51.408828
Calcite – CaCO3,21.466455
Alite_Sum C3S tot,17.053846
Calce libera,13.503319
C3A tot,3.365655
Anidrite,1.373121
Emiidrato,0.651205
SO3,0.621395
Langbeinite – MgK2(SO4)2,0.255695
Quartz,0.010952


<IPython.core.display.Javascript object>

In [26]:
pd.concat(results_to_save).groupby(
    ["Features", "Model", "Cross Validation", "Cross Validation Params"]
)[["RMSE Test", "MAE Test", "MAPE Test", "R2 Test"]].agg(
    ["mean", lambda series: pd.Series(series.std(ddof=0), name="std")]
).reset_index().rename(
    columns={"<lambda_0>": "std"}
)

Features              Model  \
                                                                       
0  Chemical + Mineralogical + Feature Engineering  Linear Regression   
1  Chemical + Mineralogical + Feature Engineering  Linear Regression   
2  Chemical + Mineralogical + Feature Engineering  Linear Regression   
3  Chemical + Mineralogical + Feature Engineering  Linear Regression   

             Cross Validation  \
                                
0  Blocking Time Series Split   
1           Out of time Split   
2              Repeated KFold   
3           Time Series Split   

                            Cross Validation Params RMSE Test            \
                                                         mean       std   
0  {"N_Splits": 5, "Repeats": 1, "train_size": 0.8}  4.745936  1.242746   
1                                {"Test Size": 0.2}  3.881371  0.000000   
2                     {"N_Splits": 5, "Repeats": 3}  2.993266  0.355498   
3           {"N_Splits": 5, "Repeats": 1, "Gap": 0}  4.283606  1.156246   

   MAE Test           MAPE Test             R2 Test            
       mean       std      mean       std      mean       std  
0  3.534024  0.328948  0.065199  0.005893 -4.029997  2.742172  
1  3.097853  0.000000  0.058789  0.000000 -1.754612  0.000000  
2  2.398074  0.223149  0.044482  0.004289  0.187197  0.121415  
3  3.103097  0.416686  0.057231  0.008904 -2.013880  1.678693

<IPython.core.display.Javascript object>

# Saving the results Dataframe

## Saving the full dataframe

In [27]:
path = "../../../../../reports/results/local_models/partner_iv/cem_a/full/"
filename = f"linear_regression_results_full_{index_to_save}.csv"

pd.concat(results_to_save).to_csv(
    path_or_buf=path + filename,
    mode="w",
    index=False,
    header=True,
)

<IPython.core.display.Javascript object>

## Saving the grouped dataframe

In [28]:
cols_groupby = [
    "Category",
    "Company",
    "Data Shape",
    "Timesteps",
    "Features",
    "Model",
    "Cross Validation",
    "Cross Validation Params",
]

cols_agg = ["RMSE Train", "MAE Train", "MAPE Train", "R2 Train"] + [
    "RMSE Test",
    "MAE Test",
    "MAPE Test",
    "R2 Test",
]

path = "../../../../../reports/results/local_models/partner_iv/cem_a/grouped/"
filename = f"linear_regression_results_grouped_{index_to_save}.csv"


df_results_to_save = (
    pd.concat(results_to_save)
    .groupby(cols_groupby, dropna=False)[cols_agg]
    .agg(["mean", lambda series: pd.Series(series.std(ddof=0), name="std")])
    .reset_index()
    .rename(columns={"<lambda_0>": "std"})
)

df_results_to_save.to_csv(
    path_or_buf=path + filename,
    mode="w",
    index=False,
    header=True,
)

<IPython.core.display.Javascript object>

In [29]:
df_results_to_save

Category Company Data Shape Timesteps  \
                                              
0  Local Model   partner_iv  (315, 28)       NaN   
1  Local Model   partner_iv  (315, 28)       NaN   
2  Local Model   partner_iv  (315, 28)       NaN   
3  Local Model   partner_iv  (315, 28)       NaN   

                                         Features              Model  \
                                                                       
0  Chemical + Mineralogical + Feature Engineering  Linear Regression   
1  Chemical + Mineralogical + Feature Engineering  Linear Regression   
2  Chemical + Mineralogical + Feature Engineering  Linear Regression   
3  Chemical + Mineralogical + Feature Engineering  Linear Regression   

             Cross Validation  \
                                
0  Blocking Time Series Split   
1           Out of time Split   
2              Repeated KFold   
3           Time Series Split   

                            Cross Validation Params RMSE Train            ...  \
                                                          mean       std  ...   
0  {"N_Splits": 5, "Repeats": 1, "train_size": 0.8}   1.272180  0.368749  ...   
1                                {"Test Size": 0.2}   2.430037  0.000000  ...   
2                     {"N_Splits": 5, "Repeats": 3}   2.520976  0.054731  ...   
3           {"N_Splits": 5, "Repeats": 1, "Gap": 0}   2.264022  0.185674  ...   

   R2 Train           RMSE Test            MAE Test           MAPE Test  \
       mean       std      mean       std      mean       std      mean   
0  0.801160  0.067343  4.745936  1.242746  3.534024  0.328948  0.065199   
1  0.533230  0.000000  3.881371  0.000000  3.097853  0.000000  0.058789   
2  0.445167  0.018597  2.993266  0.355498  2.398074  0.223149  0.044482   
3  0.609266  0.061436  4.283606  1.156246  3.103097  0.416686  0.057231   

              R2 Test            
        std      mean       std  
0  0.005893 -4.029997  2.742172  
1  0.000000 -1.754612  0.000000  
2  0.004289  0.187197  0.121415  
3  0.008904 -2.013880  1.678693  

[4 rows x 24 columns]

<IPython.core.display.Javascript object>